# DRfC (Deepracer for Cloud)

## 개요

- DRfC(Deepracer for Cloud)는 딥레이서를 콘솔이 아닌 클라우드 및 로컬에서 훈련시키는 것을 말합니다.
- 이러한 방식은 훈련 비용 절감 및 자유도가 높은 훈련 환경을 제공합니다.
- 특히, 이 강의에서는 AWS Spot Instance를 활용한 모델 훈련 방법에 대해 소개합니다.
  - Spot Instance 란, 경매 방식으로 EC2의 남는 자원을 저렴한 비용으로 이용하는 것을 말합니다.
- AWS EC2, S3, Ubuntu, Docker 등 다양한 배경지식이 필요로 합니다.

### 관련 Link

- [DRfC 메인 사이트](https://aws-deepracer-community.github.io/deepracer-for-cloud/) 
- [DRfC Docker](https://hub.docker.com/u/awsdeepracercommunity)
- [spot instacne 비용](https://aws.amazon.com/ko/ec2/spot/pricing)

### AWS instance 정보 비교

|type|spot price (변동)|vCPU|Memory|GPU|동시훈련 가능 수|
|----|----------|----|------|---|------|
|c5.2xlarge|$0.173|8|16GB||1개|
|c5.4xlarge|$0.268|16|32GB||3개|
|g4dn.2xlarge|$0.2256|8|32GB|T4 16GB|1개|
|g4dn.4xlarge|$0.3612|16|64GB|T4 16GB|3개|
|g4dn.8xlarge|$0.6528|32|128GB|T4 16GB|8개|

### Service Quotas
- GPU 인스턴스를 활용하면 훈련 속도가 훨씬 더 빠릅니다.
- 그러나 AWS 개인 계정의 경우 GPU 인스턴스를 spot(경매) 방식으로 요청할 수 있는 할당량이 주어져있지 않습니다.
- 만약, 개인 게정에서 GPU 인스턴스를 spot(경매) 요청을 하고자 한다면 Service Quotas 에서 할당량 증가를 요청해야 합니다.
  - Service Quotas > AWS 서비스 > EC2 > All G and VT Spot Instance Requests > 할당량 증가 요청 > 할당량 값 변경 (32개 정도면 충분)
- 할당량 증가 요청을 하면 검토 후 승인이 나야합니다. 그러나 승인이 거절되는 경우가 많습니다.
- 따라서 이 강의에서는 CPU 인스턴스를 활용하여 실습을 진행합니다.

### 사전 준비 사항

- S3 버킷 생성 (2개) --region us-east-1
  - local 용도의 버킷 (예) drfc-....-local
  - upload 용도의 버킷 (예) drfc-....-upload
- EC2 Role 생성
  - S3 FullAccess
- Security Group 생성
  - SSH : 22
  - Web Monitoring : 8080-8100
  - ~~VNC : 5900-5910~~
  - ~~Jupyter : 8888~~
- 키페어 생성

## 1. AMI 등록

- 기본적으로 설치해야 할 것들이 많습니다.
- 매번 인스턴스를 켜서 새로 설치하는 것은 번거로운 일이기 때문에 AMI 를 등록해 놓는 것이 좋습니다.
- 특히, Spot(경매)의 경우 언제든 인스턴스가 종료 될 수 있기 때문에 이러한 과정은 반드시 필요합니다.

### (1-1) 인스턴스 생성
  - 리전 : us-east-1 (버지니아 북부)
  - AMI : ubuntu20.04
  - 보안그룹 포트
    - SSH : 22
    - VNC : 5900-5910
    - Web Monitoring : 8080-8100
    - Jupyter : 8888
  - 스토리지 : 40 GB gp2
  - Spot 인스턴스
  - EC2 Role : S3 FullAccess  


### (1-2) 인스턴스 ssh 연결

- `EC2 인스턴스 연결`을 통해 웹브라우저 상에서 ssh 접속이 가능합니다.

### (1-3) 필수 구성요소 설치  (로컬에서는 하지 않는게 좋음)
- ```bash
  # 사용자 홈 디렉토리 이동
  cd ~

  # DRfC git 레파지토리 클론
  git clone https://github.com/aws-deepracer-community/deepracer-for-cloud.git

  # 작어폴더 이동
  cd ~/deepracer-for-cloud

  # 드라이브 분할 및 필수 구성 요소 설치 
  ./bin/prepare.sh
  ```

### (1-4)  재부팅
- ```bash
  sudo reboot now
  ```

### (1-5) 인스턴스 ssh 재연결

### (1-6) 전체 리포지토리 설정 및 핵심 Docker 이미지 다운로드 
- ```bash
  # 작업폴더 이동
  cd ~/deepracer-for-cloud

  # 전체 리포지토리 설정 및 핵심 Docker 이미지 다운로드 
  ./bin/init.sh -c aws -a cpu

  ```
  - 인스턴스 옵션 : `cpu` 또는 `gpu`

### (1-7) 유틸리티 설치
- tmux (백그라운드 다중 터미널 도구)
  ```bash
  # nmon 설치
  sudo apt install tmux
  ```

- nmon (Linux 운영 체제용 컴퓨터 성능 시스템 모니터 도구)
  ```bash
  # nmon 설치
  sudo apt install nmon
  ```

- gpustat 설치 (GPU 모니터링)
  ```bash
  # pip 인스톨
  sudo apt install -y python3-pip
  ```
  ```bash
  # gpustat 설치
  pip3 install gpustat
  ```

### (1-8) AMI 등록 및 인스턴스 종료

- (1-5)에서 cpu 옵션일 때와 gpu 옵션일 때 설치 구성요소가 다릅니다. 
- 따라서 cpu를 사용할 때와 gpu를 사용할 때 AMI 를 따로 등록해야 합니다.
- AMI 등록이 완료되면 현재 인스턴스는 종료합니다.

## 2. Environment Setup

- 우선, 등록한 AMI 로 Spot 인스턴스를 생성합니다.

### (2-1) ubuntu 사용자 전환  
- ```bash
  su - ubuntu

  ```

### (2-2) 작업폴더 이동 및 명령어 가져오기
- ```bash
  # 작업폴더이동
  cd ~/deepracer-for-cloud/

  # dr-* 명령어 가져오기
  source bin/activate.sh

  ```



### (2-3) S3 버킷 설정
- ```bash
  ############## 주요 환경변수 설정 ##############
  # S3 버킷 (로컬 및 업로드)
  DR_LOCAL_S3_BUCKET=drfc-....-local
  DR_UPLOAD_S3_BUCKET=drfc-....-upload

  # # 버킷 생성
  # aws s3 mb s3://${DR_LOCAL_S3_BUCKET} --region us-east-1
  # aws s3 mb s3://${DR_UPLOAD_S3_BUCKET} --region us-east-1

  # 환경변수 등록
  sed -i "s/\(^DR_LOCAL_S3_BUCKET=\)\(.*\)/\1$DR_LOCAL_S3_BUCKET/" system.env
  sed -i "s/\(^DR_UPLOAD_S3_BUCKET=\)\(.*\)/\1$DR_UPLOAD_S3_BUCKET/" system.env

  ```



### (2-4) 모델 목록 확인
- ```bash
  ############## 모델 목록 확인 ##############
  aws s3 ls s3://${DR_LOCAL_S3_BUCKET}

  ```



### (2-5) 모델 설정
- ```bash
  # 모델명 (폴더명)
  DR_LOCAL_S3_MODEL_PREFIX=mymodel-2

  # 모델 Clone
  DR_LOCAL_S3_PRETRAINED=False  # Clone 여부
  DR_LOCAL_S3_PRETRAINED_PREFIX=mymodel-1 # Clone 할 모델명

  # 동시 작업
  DR_WORKERS=3 # 동시 훈련 로보메이커 수

  # 다중 작업 구성
  DR_DOCKER_STYLE=compose # docker compose or swarm

  # 환경변수 등록
  sed -i "s/\(^DR_LOCAL_S3_MODEL_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_MODEL_PREFIX/" run.env
  sed -i "s/\(^DR_LOCAL_S3_PRETRAINED=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED/" run.env
  sed -i "s/\(^DR_LOCAL_S3_PRETRAINED_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED_PREFIX/" run.env
  sed -i "s/\(^DR_DOCKER_STYLE=\)\(.*\)/\1$DR_DOCKER_STYLE/" system.env
  sed -i "s/\(^DR_WORKERS=\)\(.*\)/\1$DR_WORKERS/" system.env

  ```


### (2-6) model_metadata.json

- ```bash
  ############# model_metadata.json ###########
  cat <<EOF >./custom_files/model_metadata.json
  {
    "action_space" : {
      "steering_angle" : {
        "high" : 30.0,
        "low" : -30.0
      },
      "speed" : {
        "high" : 1.0,
        "low" : 0.5
      }
    },
    "sensor": ["FRONT_FACING_CAMERA"],
    "neural_network": "DEEP_CONVOLUTIONAL_NETWORK_SHALLOW",
    "training_algorithm": "clipped_ppo", 
    "action_space_type": "continuous",
    "version": "4"
  }
  EOF

  ```

### (2-7) reward_functino.py

- ```bash
  ############## reward_function.py ###############
  cat <<EOF >./custom_files/reward_function.py
  def reward_function(params):
      # Read input parameters
      track_width = params['track_width']
      distance_from_center = params['distance_from_center']
      
      # Calculate 3 markers that are at varying distances away from the center line
      marker_1 = 0.1 * track_width
      marker_2 = 0.25 * track_width
      marker_3 = 0.5 * track_width
      
      # Give higher reward if the car is closer to center line and vice versa
      if distance_from_center <= marker_1:
          reward = 1.0
      elif distance_from_center <= marker_2:
          reward = 0.5
      elif distance_from_center <= marker_3:
          reward = 0.1
      else:
          reward = 1e-3  # likely crashed/ close to off track
      return float(reward)
  EOF

  ```

### (2-8) hyperparameters.json

- ```bash
  ############# hyperparameters.json ############
  cat <<EOF >./custom_files/hyperparameters.json
  {
      "batch_size": 128,
      "beta_entropy": 0.01,
      "discount_factor": 0.999,
      "e_greedy_value": 0.05,
      "epsilon_steps": 10000,
      "exploration_type": "categorical",
      "loss_type": "huber",
      "lr": 0.00015,
      "num_episodes_between_training": 60,
      "num_epochs": 10,
      "stack_size": 1,
      "term_cond_avg_score": 100000.0,
      "term_cond_max_episodes": 100000,
      "sac_alpha": 0.2
    }
  EOF

  ```
  - 주의 : `num_episodes_between_training` 는 모든 Worker에 대한 값이다. 단일 작업에서 설정한 값에 워커 수만큼 곱해줘야 한다.
  - 추가적으로 run.env 에서 `DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST` 환경변수는 아래와 같이 값을 맞춰줘야 한다.  
  `num_episodes_between_training / DR_WORKERS`



### (2-9) worker 설정
- ```bash
  ################ Worker 설정 ################
  #### 메인 worker ####
  DR_TRAIN_MULTI_CONFIG=False # 서브 Worker 따로 환경 설정할 지 여부

  DR_WORLD_NAME=FS_June2020 # 트랙명
  DR_RACE_TYPE=TIME_TRIAL # 레이싱 유형

  sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" run.env
  sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" run.env
  sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" run.env

  ```
  - 트랙명은 아래 사이트에서 참조 (`트랙명.npy`)  
    <https://github.com/aws-deepracer-community/deepracer-race-data/tree/main/raw_data/tracks>

## 3. RUN

### tmux
- 백그라운드 및 다중 터미널 분할
  ```bash
  # tmux 실행
  tmux

  ```
  
- tmux 명령어 및 단축키 
  - `tmux new -s <session-name>` : 지정한 세션 명으로 세션 시작
  - `tmux ls` : 세션 목록 확인
  - `tmux attach -t <session-name>` : 세션 접속
  - `ctrl+b`, `%` : 터미널 세로 분할
  - `ctrl+b`, `"` : 터미널 가로 분할
  - `ctrl+b`, `d` : 세션 빠져나오기

### 실행

- ```bash
  # configure 업데이트
  dr-update

  # custom 업데이트
  dr-upload-custom-files

  # 혹시 기존에 실행중인 터미널 있다면 종료하기
  dr-stop-training 

  # 훈련 시작
  dr-start-training -w

  ```
  
  - `-w` (wipes) 옵션은 모델명이 있는 폴더에 데이터가 존재하면 제거하는 옵션
  - `sagemaker is not running` 에러 발생 시
    - 일시적 오류이다. 아래 명령어를 통해 다시 시도한다.
    - `dr-stop-training && dr-start-training -w`
  - `Selected path s3://your_bucket/your_model. Delete it, or use -w option. Exiting.` 에러 발생 시
    - 버킷안에 이미 지정한 모델명으로 된 폴더가 존재하여 발생하는 에러.
    - 해당 모델명 폴더를 삭제하거나 `-w` 옵션을 줘서 자동으로 삭제되도록 한다.


### robomaker 모니터링

- 개별 모니터링
  -  개별 포트는 8080~8089 사이로 자동 할당 됨
  - `docker ps` 명령어를 통해 할당 된 포트를 확인 가능
  - 개별 모니터링 접속 : `http://ip주소:개별port`
  
- 전체 모니터링
  - robomaker 모니터링 컨테이너 실행
    ```bash
    # 뷰어 중지 (기존에 떠있는 뷰어 컨테이너 종료)
    dr-stop-viewer

    # 뷰어 컨테이너 실행
    dr-start-viewer

    ```

  - 전체 모니터링 접속: `http://ip주소:8100`
      - `http://ip주소:8100/?robo=all&camera=kvs_stream&quality=75&width=480`


### 기타

- `dr-increment-training -f`
  - 모델명 뒤의 `-숫자`를 하나 더하여 클론 훈련 진행
  - (예) 현재 모델명이 `test-1` 으로 훈련중일 때, 아래와 같이 명령어를 작성하면 `test-2`로 클론한 모델이 훈련 됨.
  
    ```bash
    dr-stop-training
    dr-increment-training -f
    dr-update && dr-upload-custom-files && dr-start-training -w
    ```

- `dr-upload-model`
  - 현재 훈련중인 마지막 모델을 업로드 버킷으로 업로드
  - `-b` 옵션을 주면 최상위 모델을 업로드
